# Suspicious SQL Error Messages
Detects SQL error messages that indicate probing for an injection attack

## Rule Content
```
- title: Suspicious SQL Error Messages
  id: 8a670c6d-7189-4b1c-8017-a417ca84a086
  status: experimental
  description: Detects SQL error messages that indicate probing for an injection attack
  author: Bjoern Kimminich
  references:
  - http://www.sqlinjection.net/errors
  logsource:
    category: application
    product: sql
    service: null
  detection:
    keywords:
    - quoted string not properly terminated
    - You have an error in your SQL syntax
    - Unclosed quotation mark
    - 'near "*": syntax error'
    - SELECTs to the left and right of UNION do not have the same number of result
      columns
    condition: keywords
  falsepositives:
  - Application bugs
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='\*.keyword:(*quoted\ string\ not\ properly\ terminated* OR *You\ have\ an\ error\ in\ your\ SQL\ syntax* OR *Unclosed\ quotation\ mark* OR *near\ \"*\"\:\ syntax\ error* OR *SELECTs\ to\ the\ left\ and\ right\ of\ UNION\ do\ not\ have\ the\ same\ number\ of\ result\ columns*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()